# Secretory Reactions
Here we will map all the secretory reactions from the [Mammalian Secretory Pathway Reconstruction paper](https://www.nature.com/articles/s41467-019-13867-y) to the genes in our reconstruction.

1. Load Secretory Pathway Reactions

In [ ]:
import re
import pandas as pd
import numpy as np
from google_sheet import GoogleSheet

import matplotlib.pyplot as plt
from matplotlib_venn import venn3

from cobra.io import load_json_model
from cobra import Model, Reaction, Metabolite

In [ ]:
import importlib
import iCHOSEC_Builder_2

from iCHOSEC_Builder_2 import iCHOSEC_Builder

In [ ]:
##### ----- Generate datasets from Google Sheet ----- #####

#Credential file
KEY_FILE_PATH = 'credentials.json'

#CHO Network Reconstruction + Recon3D_v3 Google Sheet ID
Sec_Recon_SPREADSHEET_ID = '1DaAdZlvMYDqb7g31I5dw-ZCZH52Xj_W3FnQMFUzqmiQ'

# Initialize the GoogleSheet object
sec_recon_gsheet_file = GoogleSheet(Sec_Recon_SPREADSHEET_ID, KEY_FILE_PATH)

# Read data from SecRecon
sec_reactions_sheet = 'Reactions'
sec_reactions = sec_recon_gsheet_file.read_google_sheet(sec_reactions_sheet)

In [ ]:
# Load model
model = load_json_model('Models/iCHO3595.json')

In [ ]:
# Sec Recon Reaction Files
rxnFormula = list(sec_reactions['Reaction Formula'])
rxnAbbreviation = list(sec_reactions['Reaction'])
rxnPathway = list(sec_reactions['Pathway'])
rxnConditions = list(sec_reactions['Conditions'])
rxnGPR = list(sec_reactions['GPR_final'])
rxnGPR = ['' if x is None else x for x in rxnGPR] # transform None values to empty string
rxnComponents = list(sec_reactions['Components'])

# CHO PSIM
PSIM = pd.read_csv('PSIM/PSIM_CHO.tab', sep='\t')
PSIM.fillna({'Location': ''}, inplace=True) # Replace NaN values in the 'Location' column with empty strings
PSIM.set_index('Entry', inplace=True)

In [ ]:
# Define the entry ID you are looking for
entryID = 'G3HGQ6'

# Create the Protein Specific Information (PSI) list
PSI = PSIM.loc[entryID]

In [ ]:
importlib.reload(iCHOSEC_Builder_2)
from iCHOSEC_Builder_2 import iCHOSEC_Builder

builder = iCHOSEC_Builder(PSIM=PSIM,
                          entryID=entryID,
                          rxnPathway=rxnPathway,
                          rxnFormula=rxnFormula,
                          rxnAbbreviation=rxnAbbreviation,
                          rxnConditions=rxnConditions,
                          rxnGPR=rxnGPR,
                          rxnComponents = rxnComponents
                         )

In [ ]:
# Generate lists for reactions (rxns), reaction names (rxnNames), and Gene-Protein-Reactions (GPRs)
rxns, rxnNames, GPRs = builder.generateProteinSpecificRxns_A()

In [ ]:
rxns

In [ ]:
sec_reactions_dict = {}
for name, rxn, gpr in zip(rxnNames, rxns, GPRs):
    sec_reactions_dict[name] = {'rxn': rxn, 'GPR': gpr}

In [ ]:
sec_reactions_dict

In [ ]:
# Initialize new model
secretory_model = model.copy()

# Add Sec Recon Reactions IDs
sec_r = []
for rxn in sec_reactions_dict.keys():
    r = Reaction(rxn)
    sec_r.append(r)
secretory_model.add_reactions(sec_r)

for r in secretory_model.reactions:
    if r in sec_r:
        r.build_reaction_from_string(sec_reactions_dict[r.id]['rxn'])
        r.gene_reaction_rule = str(sec_reactions_dict[r.id]['GPR'])
        print(r.reaction)
        print(f"Lower bound: {r.lower_bound}, Upper bound: {r.upper_bound}")

In [ ]:
# Set the objective to secretion of target protein
sreaction = Reaction('DM_G3HGQ6_c')
secretory_model.add_reactions([sreaction])
sreaction.name = 'Demand for G3HGQ6 '
sreaction.lower_bound = 0.
sreaction.upper_bound = 1000.
sreaction.build_reaction_from_string('G3HGQ6_c -->')
secretory_model.add_reactions([sreaction])

In [ ]:
secretory_model.objective = 'DM_G3HGQ6_c'
secretory_model.slim_optimize() # mmol of target protein/gDW/h

In [ ]:

# Plot Pareto frontier of secretion-for-growth trade-off
secretory_model.objective = 'biomass_cho_prod'
max_growth = secretory_model.slim_optimize()
secretory_model.objective = entryID + '_Final_demand'
steps = 50
x = [max_growth/steps * i for i in range(steps)]
y = [0] * len(x)
for n in range(steps):
    mu_constraint = x[n]
    secretory_model.reactions.get_by_id('biomass_cho_prod').lower_bound = mu_constraint
    y[n] = secretory_model.slim_optimize()
plt.plot(x,y,linewidth=3,label=entryID)
plt.legend()
plt.xlabel('$\mu$ [1/h]')
plt.ylabel('qP [mmol/gDW/h]')
plt.title('Productivity-for-growth trade-off')